In [1]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [2]:
documents[2]

{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
 'section': 'General course-related questions',
 'question': 'Course - Can I still join the course after the start date?',
 'course': 'data-engineering-zoomcamp'}

In [3]:
import minsearch

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [4]:
from groq import Groq

groq_client = Groq()

In [5]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [6]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [7]:
def llm(prompt):
    response = groq_client.chat.completions.create(
        model='qwen/qwen3-32b',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content


In [8]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [9]:
rag('how do I run kafka?')

'<think>\nOkay, the user is asking how to run Kafka. Let me check the provided CONTEXT for relevant information. \n\nFirst, looking at the sections related to running Kafka. There\'s a part about Java Kafka where they mention running producer/consumer using a Java command. The command is java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java. That\'s for Java-based Kafka applications.\n\nThen there\'s a Python-related issue where the module “kafka” wasn\'t found. The solution involves creating a virtual environment, installing requirements from requirements.txt. Commands like python -m venv env, activating the environment, and installing dependencies. Also, notes about Windows paths being different. This would apply if the user is using Python scripts for Kafka.\n\nAnother part mentions fixing a Permission denied error on build.sh by using chmod +x build.sh. That\'s about making the script executable. And there\'s a ModuleNotFoundError related t

In [10]:
# Rag with vector search

In [11]:
from qdrant_client import QdrantClient, models

In [12]:
qd_client = QdrantClient("http://localhost:6333")

In [13]:
EMBEDDING_DIMENSIONALITY = 512
model_handle = "jinaai/jina-embeddings-v2-small-en"

In [14]:
collection_name = "zoomcamp-faq"

In [ ]:
#qd_client.delete_collection(collection_name=collection_name)

In [15]:
qd_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,
        distance=models.Distance.COSINE
    )
)

True

In [16]:
points = []

for i, doc in enumerate(documents):
    text = doc['question'] + ' ' + doc['text']
    vector = models.Document(text=text, model=model_handle)
    point = models.PointStruct(
        id=i,
        vector=vector,
        payload=doc
    )
    points.append(point)

In [17]:
points[0]

PointStruct(id=0, vector=Document(text="Course - When will the course start? The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.", model='jinaai/jina-embeddings-v2-small-en', options=None), payload={'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with ann

In [18]:
qd_client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [19]:
question = 'I just discovered the course. Can I still join it?'

In [20]:
def vector_search(question):
    print('vector_search is used')
    
    course = 'data-engineering-zoomcamp'
    query_points = qd_client.query_points(
        collection_name=collection_name,
        query=models.Document(
            text=question,
            model=model_handle 
        ),
        query_filter=models.Filter( 
            must=[
                models.FieldCondition(
                    key="course",
                    match=models.MatchValue(value=course)
                )
            ]
        ),
        limit=5,
        with_payload=True
    )
    
    results = []
    
    for point in query_points.points:
        results.append(point.payload)
    
    return results

In [21]:
def rag(query):
    search_results = vector_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [22]:
rag('how do I run kafka?')

vector_search is used


"<think>\nOkay, the user is asking how to run Kafka. Let me check the CONTEXT provided.\n\nFirst, there's a section on running Java Kafka producers, consumers, etc. The answer mentions using a Java command with the JAR file. Then there are solutions for errors like no brokers available, which relates to Docker setup. Also, there's a Python part about setting up a virtual environment for Kafka in Python.\n\nThe user's question is general, but the CONTEXT is split between Java and Python. Since Kafka itself can be run via Docker, maybe they need to start the Docker containers first. The error solutions mention Docker, so that's a key step. \n\nThe user might be new and not know the prerequisites. So the answer should probably include starting the Docker setup first using docker-compose. Then, for Java, the specific command to run the producer or consumer. Also, for Python scripts, using a virtual environment as described. \n\nI need to make sure the answer covers all necessary steps: Doc